In [94]:
import pandas as pd
import numpy as np
from pyalgotrade import strategy, plotter
from pyalgotrade.technical import bollinger
from pyalgotrade.stratanalyzer import returns, sharpe, trades, drawdown
from pyalgotrade.barfeed import csvfeed
from pyalgotrade.bar import Frequency
from pyalgotrade.broker import backtesting, slippage
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

<div class="jumbotron">
    <p class="display-1 h1">实盘交易</p>
    <hr class="my-4">
    <p>主讲：李岩</p>
    <p>管理学院</p>
    <p>liyan@cumtb.edu.cn</p>
</div>

- 实盘交易需要考虑的问题
    - 交易成本
    - 风险控制
    - 交易系统

# 交易成本

- 交易成本（transaction cost）：买卖标的品种所需要的成本

- 分类
    - 显性交易成本：在交易之前可以准确测算
    - 隐性交易成本：在交易过程中由于市场环境等因素的不确定性产生

## 显性交易成本

### 手续费

- 手续费（commissions and fees）：交给经纪商（brokerages）和交易所（exchanges）的费用

- 期货交易手续费有两种表示方法

1. 交易金额的固定比例，比如，`rb`的手续费率为0.1&permil;

- `rb2101`的开仓成交价为3627元，则手续费为$3627\times 10\times 0.1$&permil;$=3.63$元

2. 每手的固定金额，比如，`ru`的手续费额为3元/手

- `rb`基于布林线的均值回归策略，累积盈利$3165$元，交易次数为$266$次
    - 一次完整交易（开平）手续费为$7.26$元，总手续费为$7.26\times 266=1931$元

### `pyalgotrade`中手续费的设置

```python
    pyalgotrade.broker.backtesting.TradePercentage(percentage)
    pyalgotrade.broker.backtesting.FixedPerTrade(amount)
```
- `percentage`：比例
- `amount`：金额

#### 设置手续费

<center><img src="./img/realTrading/commissionSetting.png" width=100%></center>

In [70]:
class myBBands(strategy.BacktestingStrategy):
    
    def __init__(self, feed, instrument, bBandsPeriod, numDev, initEquity, lots):
        super(myBBands, self).__init__(feed, initEquity)
        self.__instrument = instrument
        self.__bbands = bollinger.BollingerBands(feed[instrument].getCloseDataSeries(), bBandsPeriod, numDev)
        self.__positivePosition = None
        self.__negativePosition = None
        self.__commission = 0
        self.__lots = lots

    def getBollingerBands(self):
        return self.__bbands
    
    def getCommission(self):
        return self.__commission
    
    def onExitOk(self, position):
        order = position.getExitOrder()
        execInfo = order.getExecutionInfo()
        self.__commission += execInfo.getCommission()
#         self.info(f'{self.action_type(order)}，价格为{execInfo.getPrice():.2f}，手续费为{execInfo.getCommission():.2f}')
        if self.__negativePosition == position:
            self.__negativePosition = None
        elif self.__positivePosition == position:
            self.__positivePosition = None
        
    def onEnterOk(self, position):
        order = position.getEntryOrder()
        execInfo = order.getExecutionInfo()
        self.__commission += execInfo.getCommission()
#         self.info(f'{self.action_type(order)}，价格为{execInfo.getPrice():.2f}，手续费为{execInfo.getCommission():.2f}')
        
    def action_type(self, order):
        actionType = order.getAction()
        typeMeaning = ''
        if actionType == 1:
            typeMeaning = '建多仓'
        elif actionType == 2:
            typeMeaning = '平空仓'
        elif actionType == 3:
            typeMeaning = '平多仓'
        elif actionType == 4:
            typeMeaning = '建空仓'
        return typeMeaning
            

    def onBars(self, bars):
        lower = self.__bbands.getLowerBand()[-1]
        upper = self.__bbands.getUpperBand()[-1]
        if lower is None:
            return

        bar = bars[self.__instrument]
        
        if bar.getClose() < lower:
            if self.__negativePosition is not None and self.__negativePosition.exitActive() is False:
                # 有空仓，并且没有平空仓的挂单
                self.__negativePosition.exitMarket()
            if self.__positivePosition is None:
                # 没有多仓
                self.__positivePosition = self.enterLong(self.__instrument, self.__lots)
        elif bar.getClose() > upper:
            if self.__positivePosition is not None and self.__positivePosition.exitActive() is False:
                # 有多仓，并且没有平多仓的挂单
                self.__positivePosition.exitMarket()
            if self.__negativePosition is None:
                # 没有空仓
                self.__negativePosition = self.enterShort(self.__instrument, self.__lots)

In [16]:
def read_data(instrument):
    strFeed = csvfeed.GenericBarFeed(Frequency.MINUTE)
    strFeed.addBarsFromCSV(instrument,'./data/performance/rb.csv')
    return strFeed

In [62]:
def main():
    instrument = 'rb'
    multiplier = 10
    bBandsPeriod = 40
    numDev = 3
    feed = read_data(instrument)
 
    # 建立比例手续费对象
    cmFee = backtesting.TradePercentage(0.001)
    slip = slippage.VolumeShareSlippage(100)
        
    strat = myBBands(feed, instrument, bBandsPeriod, numDev, 50000, 5)
    
    # 给策略对应的broker设置手续费
    strat.getBroker().setCommission(cmFee)
    strat.run()
    
    finalEquity = strat.getResult()
    
    print(f'最终权益为：{finalEquity:.2f}')
    print(f'手续费为：{strat.getCommission():.2f}')
    print(f'最终权益(不考虑手续费)为：{finalEquity+strat.getCommission():.2f}')

In [58]:
main()

2019-12-05 14:55:00 strategy [INFO] 建多仓，价格为3412.00，手续费为17.06
2019-12-06 13:54:00 strategy [INFO] 平多仓，价格为3434.00，手续费为17.17
2019-12-06 13:54:00 strategy [INFO] 建空仓，价格为3434.00，手续费为17.17
2019-12-10 14:15:00 strategy [INFO] 平空仓，价格为3507.00，手续费为17.54
2019-12-10 14:15:00 strategy [INFO] 建多仓，价格为3507.00，手续费为17.54
2019-12-10 21:03:00 strategy [INFO] 平多仓，价格为3516.00，手续费为17.58
2019-12-10 21:03:00 strategy [INFO] 建空仓，价格为3516.00，手续费为17.58
2019-12-10 22:08:00 strategy [INFO] 平空仓，价格为3511.00，手续费为17.55
2019-12-10 22:08:00 strategy [INFO] 建多仓，价格为3511.00，手续费为17.55
2019-12-10 22:27:00 strategy [INFO] 平多仓，价格为3529.00，手续费为17.64
2019-12-10 22:27:00 strategy [INFO] 建空仓，价格为3529.00，手续费为17.64
2019-12-11 09:36:00 strategy [INFO] 平空仓，价格为3530.00，手续费为17.65
2019-12-11 09:36:00 strategy [INFO] 建多仓，价格为3530.00，手续费为17.65
2019-12-12 13:31:00 strategy [INFO] 平多仓，价格为3523.00，手续费为17.62
2019-12-12 13:31:00 strategy [INFO] 建空仓，价格为3523.00，手续费为17.62
2019-12-12 21:08:00 strategy [INFO] 平空仓，价格为3518.00，手续费为17.59
2019-12-12 21:08:00 stra

最终权益为：43711.21
手续费为：9453.79
最终权益(不考虑手续费)为：53165.00


## 隐性交易成本

### 价格冲击

- 某一订单提交到市场后，引起交易品种价格的变化

- 如果建立某个交易品种的**多头**仓位，该交易品种的价格通常会**上涨**
- 如果建立某个交易品种的**空头**仓位，该交易品种的价格通常会**下跌**

- 价格冲击通常受到**订单大小**与下单时市场上的**流动性**之间的相对关系的影响

### 滑点

- 滑点（slippage）：决定交易时候的市场价格与订单被真正执行时候市场价格之间的差，又被称作“延误成本”

<center><img src="./img/realTrading/slippage.png" width=100%></center>

- 多单的期望建仓价格为$100$元，但是实际执行的价格是$105$元，$5$元即为滑点

#### 影响滑点大小的因素

- 趋势追踪策略易于受到滑点的负面影响

- 价格波动越剧烈，越易出现滑点

- 通信延迟越大，越易出现滑点

### `pyalgotrade`滑点设置

```python
    pyalgotrade.broker.slippage.VolumeShareSlippage(priceImpact=0.1)
```
- `priceImpact`：下单规模对市场价格影响的程度

#### 原理

$$
P=P_0(1+MI)
$$
- $P$：考虑滑点后的价格
- $P_0$：希望的成交价格
- $MI$：价格对市场的冲击

$$
MI = \text{priceImpactConst}\times \bigl(\frac{\text{订单规模}}{\text{总成交量}}\bigr)^2
$$

#### 使用滑点设置

<center><img src="./img/realTrading/slippageSetting.png" width=100%></center>

In [85]:
def main_slippage(inputEquity, inputLots, slippageConst):
    instrument = 'rb'
    multiplier = 10
    bBandsPeriod = 40
    numDev = 3
    feed = read_data(instrument)
 
    # 建立比例手续费对象
#     cmFee = backtesting.TradePercentage(0.001)
    slip = slippage.VolumeShareSlippage(slippageConst)
        
    strat = myBBands(feed, instrument, bBandsPeriod, numDev, inputEquity, inputLots)
    
    # 给策略对应的broker设置手续费
#     strat.getBroker().setCommission(cmFee)

    # 设置滑点
    strat.getBroker().getFillStrategy().setSlippageModel(slip)
    strat.run()
    
    finalEquity = strat.getResult()
    return finalEquity

In [80]:
print(f'最终权益为：{main_slippage(50000,5,0.01):.2f}')

最终权益为：53165.00


#### 价格影响因子的作用

In [101]:
equityLst = []
priceImpactLst = [0.1, 10, 100, 1000]
for each in priceImpactLst:
    equityLst.append(main_slippage(500000,50, each))

df = pd.DataFrame({'影响因子':priceImpactLst, '最终权益':equityLst})
df

,影响因子,最终权益
0,0.1,531644.097779
1,10.0,531059.777875
2,100.0,525747.778750
3,1000.0,472627.787500


#### 订单规模的影响

In [99]:
orderSizeLst = [10, 50, 100]
equityLst1 = []
for each in orderSizeLst:
    equityLst1.append(main_slippage(500000, each, 100))
    
df1 = pd.DataFrame({'订单规模':orderSizeLst, '最终权益':equityLst1})
df1

,订单规模,最终权益
0,10,506282.78223
1,50,525747.77875
2,100,516082.23000


# 交易系统

## 国内期货市场的交易系统

- 交易所系统
- 经纪商系统
- 投资者系统

- 交易所系统：交易撮合队列、行情服务器

- 经纪商柜台系统：以CTP为代表，
    - 用于接收客户订单、前端风控
    - 接收交易所推送的行情

- 投资者系统
    - 连接柜台、行情服务器
    - 启动和关闭交易策略

### 三套系统之间的交互

<center><img src="./img/realTrading/systems.svg" width=100%></center>

### CTP接口

- 综合交易平台（Comprehensive Transaction Platform）
    - 由上海期货交易所的上海期货信息技术有限公司开发 [http://www.sfit.com.cn/](http://www.sfit.com.cn/)
    - 适用于期货公司的期货经纪业务管理系统

- CTP的组成
<center><img src="./img/realTrading/ctp.png" width=100%></center>

- CTP能够同时连通上海期货交易所、大连商品交易所、郑州商品交易所、中国金融期货交易所
- 支持国内商品期货和股指期货的交易和结算业务

- 分别对应交易接口（`Trader API`）、风控接口(`Risk API`)、结算接口（`CSV`）

### `Trader API`

- 最新版本是v6.3.19_P1

文件名|功能说明
---|---
`ThostFtdcTraderApi.h`|`C++`头文件，包含交易相关的指令
`ThostFtdcMdApi.h`|`C++`头文件，包含获取行情相关的指令
`ThostFtdcUserApiStruct.h`|包含所有需要的数据结构
`ThostFtdcUserApiDataType.h`|包含所有需要用到的数据类型
`thosttraderapi_se.dll`|交易相关的动态链接库
`thosttraderapi_se.lib`|交易相关的静态链接库
`thostmduserapi_se.dll`|行情相关的动态链接库
`thostmduserapi_se.lib`|行情相关的静态链接库
`error.dtd`|所有可能的错误信息
`error.xml`|所有可能的错误信息

- 用户只需要关注`.h`文件

#### `API`接口包含的两个***类***

- `Spi`类：包含所有的响应和回报函数，接收CTP发送的消息

- `Api`类：用户主动向CTP发起请求和订阅的函数

<center><img src="./img/realTrading/ctpApiSpi.svg" width=100%></center>

## 国内量化交易平台

- 优矿 [https://uqer.datayes.com/](https://uqer.datayes.com/)
- BigQuant [https://bigquant.com/](https://bigquant.com/)
- 聚宽 [https://www.joinquant.com/](https://www.joinquant.com/)
- 掘金量化 [https://www.myquant.cn/](https://www.myquant.cn/)

### 构建一个交易策略主要使用的函数

```python
    def initialize(context):
        # 初始化函数，初始化策略的基本参数
        pass
    
    def handle_data(context, data):
        # 主交易逻辑
        pass
```